In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np

from keras.applications import vgg16
from keras.preprocessing.image import load_img,img_to_array
from keras.models import Model
from keras.applications.imagenet_utils import preprocess_input


from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np # linear algebra
import pandas as pd 
import matplotlib.pyplot as plt
import os

In [ ]:
%cd /kaggle/input/fashion-product-images-dataset/fashion-dataset/fashion-dataset/

In [ ]:
sdf=pd.read_csv('styles.csv',error_bad_lines=False)

In [ ]:
sdf['articleType']=sdf['articleType'].str.lower()
sdf['id']=sdf['id'].astype(str)
sdf=sdf[sdf['articleType'].str.find('shirt')==True]
sdf=sdf.reset_index()
sdf.drop(columns=['index'],inplace=True)
sdf.dropna(subset=['season'],inplace=True)
sdf['season'].unique()


In [ ]:
type(sdf['id'][0])
image_list=list(sdf['id'])
imgs_path = "/kaggle/input/fashion-product-images-dataset/fashion-dataset/fashion-dataset/images/"
imgs_model_width, imgs_model_height = 224, 224
nb_closest_images = 5

In [ ]:
files=[]
for x in image_list:
  files.append(imgs_path+x+".jpg")

In [ ]:
files[0]

In [ ]:
files=files[0:3000]
top_images=pd.DataFrame(files[-200:],columns=['image_id'])
# top_images.to_csv('top_images.csv',index=False)
print(len(files))
print(len(top_images))

In [ ]:
original = load_img(files[9], target_size=(imgs_model_width, imgs_model_height))
plt.imshow(original)
plt.show()
print("Image loaded successfully!")

In [ ]:
# load the model
vgg_model = vgg16.VGG16(weights='imagenet')

# remove the last layers in order to get features instead of predictions
feat_extractor = Model(inputs=vgg_model.input, outputs=vgg_model.get_layer("fc2").output)

# print the layers of the CNN
feat_extractor.summary()

In [ ]:
numpy_image = img_to_array(original)
# convert the image / images into batch format
# expand_dims will add an extra dimension to the data at a particular axis
# we want the input matrix to the network to be of the form (batchsize, height, width, channels)
# thus we add the extra dimension to the axis 0.
image_batch = np.expand_dims(numpy_image, axis=0)
print('Image Batch size', image_batch.shape)

# prepare the image for the VGG model
processed_image = preprocess_input(image_batch.copy())

In [ ]:
img_features = feat_extractor.predict(processed_image)

print("Features successfully extracted for one image!")
print("Number of image features:",img_features.size)
img_features

In [ ]:
importedImages = []

for f in files:
    filename = f
    original = load_img(filename, target_size=(224, 224))
    numpy_image = img_to_array(original)
    image_batch = np.expand_dims(numpy_image, axis=0)
    
    importedImages.append(image_batch)
    
images = np.vstack(importedImages)

processed_imgs = preprocess_input(images.copy())

In [ ]:
%cd 

In [ ]:
top_images.to_csv('/kaggle/working/top_images.csv',index=False)

In [ ]:
%cd /kaggle/input/fashion-product-images-dataset/fashion-dataset/fashion-dataset/

In [ ]:
imgs_features = feat_extractor.predict(processed_imgs)

print("features successfully extracted!")
imgs_features.shape

In [ ]:
cosSimilarities = cosine_similarity(imgs_features)

# store the results into a pandas dataframe

cos_similarities_df = pd.DataFrame(cosSimilarities, columns=files, index=files)
cos_similarities_df.head()

In [ ]:
%cd 

In [ ]:
cos_similarities_df.to_csv("/kaggle/working/cos_similarity.csv",index=False)

In [ ]:
%cd /kaggle/input/fashion-product-images-dataset/fashion-dataset/fashion-dataset/

In [ ]:
# function to retrieve the most similar products for a given one
def retrieve_most_similar_products(given_img):

    print("-----------------------------------------------------------------------")
    print("original product:")

    original = load_img(given_img, target_size=(imgs_model_width, imgs_model_height))
    plt.imshow(original)
    plt.show()

    print("-----------------------------------------------------------------------")
    print("most similar products:")

    closest_imgs = cos_similarities_df[given_img].sort_values(ascending=False)[1:nb_closest_images+1].index
    closest_imgs_scores = cos_similarities_df[given_img].sort_values(ascending=False)[1:nb_closest_images+1]

    for i in range(0,len(closest_imgs)):
        original = load_img(closest_imgs[i], target_size=(imgs_model_width, imgs_model_height))
        plt.imshow(original)
        plt.show()
        print("similarity score : ",closest_imgs_scores[i])

In [ ]:
retrieve_most_similar_products(top_images['image_id'][16])

In [ ]:
retrieve_most_similar_products(top_images['image_id'][81])

In [ ]:
retrieve_most_similar_products(top_images['image_id'][47])

In [ ]:
retrieve_most_similar_products(top_images['image_id'][60])

In [ ]:
retrieve_most_similar_products(top_images['image_id'][168])

In [ ]:
retrieve_most_similar_products(top_images['image_id'][77])